In [1]:
import numpy as np
import os

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import time

In [3]:
import tensorflow as tf

In [2]:
def reset_graph(seed_=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed_)
    np.random.seed(seed_)

In [21]:
reset_graph()

x = tf.Variable(3, name = 'x')
y = tf.Variable(4, name = 'y')
a = tf.constant(2, name = 'a')

In [22]:
print(x.graph is tf.get_default_graph())
print(y.graph is tf.get_default_graph())
print(a.graph is tf.get_default_graph())

True
True
True


In [23]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
False


In [24]:
x, y, a

(<tf.Variable 'x:0' shape=() dtype=int32_ref>,
 <tf.Variable 'y:0' shape=() dtype=int32_ref>,
 <tf.Tensor 'a:0' shape=() dtype=int32>)

In [25]:
f = x*x*y + y + a

In [26]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [27]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
a_val = a.eval(session=sess)
print('a =', a_val)
print('result =', result)
sess.close()

a = 2
result = 42


In [28]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    a_val = a.eval()

print('a =', a_val)
print('result =', result)

a = 2
result = 42


In [30]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

42


In [31]:
init

<tf.Operation 'init_1' type=NoOp>

In [33]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [34]:
x

<tf.Tensor 'add_5:0' shape=() dtype=int32>

In [35]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


Definig a composite function f(w) = exp( w20 + w21 * exp( w10 + w11 * exp( w00 + w01*x ) ) )

In [37]:
def my_func(w, x):
    f_0 = tf.exp(w[0, 0] + w[0, 1]*x)
    f_1 = tf.exp(w[1, 0] + w[1, 1]*f_0)
    f_2 = tf.exp(w[2, 0] + w[2, 1]*f_1)

    return f_2, f_1, f_0

In [38]:
w_0 = np.vstack( ( np.zeros(3), np.ones(3) ) ).T

In [39]:
w_0

array([[0., 1.],
       [0., 1.],
       [0., 1.]])

In [42]:
reset_graph()

w = tf.Variable(w_0, name='w', dtype=tf.float32)
x = tf.Variable(1.0, name='x', dtype=tf.float32, trainable=False)

f_2, f_1, f_0 = my_func(w, x)
grads = tf.gradients(f_2, w)

grads

[<tf.Tensor 'gradients/AddN:0' shape=(3, 2) dtype=float32>]

In [68]:
init = tf.global_variables_initializer()

t_0 =time.time()

with tf.Session() as sess:
    sess.run(init)
    gradients, function_vals = sess.run([grads, [f_2, f_1, f_0]])

print('%f sec' % (time.time()-t_0))
print(function_vals)
print(gradients)

0.026084 sec
[3814273.0, 15.154261, 2.7182817]
[array([[1.5712344e+08, 1.5712344e+08],
       [5.7802488e+07, 1.5712344e+08],
       [3.8142730e+06, 5.7802488e+07]], dtype=float32)]


In [69]:
my_func(w_0, x)

(<tf.Tensor 'Exp_5:0' shape=() dtype=float32>,
 <tf.Tensor 'Exp_4:0' shape=() dtype=float32>,
 <tf.Tensor 'Exp_3:0' shape=() dtype=float32>)